# Airline delays 
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

~140GB

In [2]:
# imports
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dbutils.fs.ls('/databricks-datasets/airlines')

Out[2]: [FileInfo(path='dbfs:/databricks-datasets/airlines/README.md', name='README.md', size=1089),
 FileInfo(path='dbfs:/databricks-datasets/airlines/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00000', name='part-00000', size=67108879),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00001', name='part-00001', size=67108862),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00002', name='part-00002', size=67108930),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00003', name='part-00003', size=67108804),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00004', name='part-00004', size=67108908),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00005', name='part-00005', size=67108890),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00006', name='part-00006', size=67108825),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00007', name='part-00007', size=67108880),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00008', name='part-00008', size=67108832),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00009', name='part-00009', size=67108857),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00010', name='part-00010', size=67108931),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00011', name='part-00011', size=67108872),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00012', name='part-00012', size=67108791),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00013', name='part-00013', size=67108856),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00014', name='part-00014', size=67108855),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00015', name='part-00015', size=67108875),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00016', name='part-00016', size=67108898),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00017', name='part-00017', size=67108873),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00018', name='part-00018', size=67108881),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00019', name='part-00019', size=67108829),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00020', name='part-00020', size=67108863),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00021', name='part-00021', size=67108868),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00022', name='part-00022', size=67108827),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00023', name='part-00023', size=67108926),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00024', name='part-00024', size=67108885),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00025', name='part-00025', size=67108843),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00026', name='part-00026', size=67108837),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00027', name='part-00027', size=67108911),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00028', name='part-00028', size=67108776),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00029', name='part-00029', size=67108929),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00030', name='part-00030', size=67108839),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00031', name='part-00031', size=67108867),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00032', name='part-00032', size=67108865),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00033', name='part-00033', size=67108864),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00034', name='part-00034', size=67108854),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00035', name='part-00035', size=67108878),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00036', name='part-00036', size=67108847),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00037', name='part-00037', size=67108895),
 FileInfo(path='dbfs:/databricks-datasets/airline

In [4]:
sum = 0
DATA_PATH = 'dbfs:/databricks-datasets/airlines/'
for item in dbutils.fs.ls(DATA_PATH):
  sum = sum+item.size
sum


Out[3]: 128911081815

In [5]:
with open("/dbfs/databricks-datasets/airlines/README.md") as f:
    x = ''.join(f.readlines())

print(x)

================================================
Airline On-Time Statistics and Delay Causes
================================================

## Background
The U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics (BTS) tracks the on-time performance of domestic flights operated by large air carriers. Summary information on the number of on-time, delayed, canceled and diverted flights appears in DOT's monthly Air Travel Consumer Report, published about 30 days after the month's end, as well as in summary tables posted on this website. BTS began collecting details on the causes of flight delays in June 2003. Summary statistics and raw data are made available to the public at the time the Air Travel Consumer Report is released.


FAQ Information is available at http://www.rita.dot.gov/bts/help_with_data/aviation/index.html


## Data Source
http://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp


## Usage Restrictions
The data is released under the Freedom of Information act. More information can be found at http://www.fas.org/sgp/foia/citizen.html

In [6]:
# airlines_first = spark.read.option("header", "true").csv("dbfs:/databricks-datasets/airlines/part-00000")
# headers = airlines_first.columns
# airlines_first.count()
#airlines = spark.read.csv("dbfs:/databricks-datasets/airlines/part-*")
#airlines.printSchema()

In [7]:
from pyspark.sql.types import *
airline_schema = StructType([
            StructField("Year", IntegerType(), True),
            StructField("Month", IntegerType(), True),
            StructField("DayofMonth", IntegerType(), True),
            StructField("DayOfWeek", IntegerType(), True),
            StructField("DepTime", IntegerType(), True),
            StructField("CRSDepTime", IntegerType(), True),
            StructField("ArrTime", IntegerType(), True),
            StructField("CRSArrTime", IntegerType(), True),
            StructField("UniqueCarrier", StringType(), True),
            StructField("FlightNum", StringType(), True),
            StructField("TailNum", StringType(), True),
            StructField("ActualElapsedTime", IntegerType(), True),
            StructField("CRSElapsedTime", IntegerType(), True),
            StructField("AirTime", IntegerType(), True),
            StructField("ArrDelay", IntegerType(), True),
            StructField("DepDelay", IntegerType(), True),
            StructField("Origin", StringType(), True),
            StructField("Dest", StringType(), True),
            StructField("Distance", IntegerType(), True),
            StructField("TaxiIn", IntegerType(), True),			
			StructField("TaxiOut", IntegerType(), True),
            StructField("Cancelled", IntegerType(), True),
            StructField("CancellationCode", StringType(), True),
            StructField("Diverted", IntegerType(), True),
            StructField("CarrierDelay", IntegerType(), True),
            StructField("WeatherDelay", IntegerType(), True),
            StructField("NASDelay", IntegerType(), True),
            StructField("SecurityDelay", IntegerType(), True),
			StructField("LateAircraftDelay", IntegerType(), True),
            StructField("IsArrDelayed", StringType(), True),			
            StructField("IsDepDelayed", StringType(), True)])

In [8]:
airlines = spark.read.format("csv").option("header", "false").schema(airline_schema).load("dbfs:/databricks-datasets/airlines/part-*")
airlines.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: integer (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: integer (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: string (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: integer (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- AirTime: integer (nullable = true)
-- ArrDelay: integer (nullable = true)
-- DepDelay: integer (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: integer (nullable = true)
-- TaxiOut: integer (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: integer (nullable = true)
-- WeatherDelay: integer (nullable = true)
-- NASDelay: integer (nullable = true)
-- SecurityDelay: integer (nullable = true)
-- LateAircraftDelay: integer (nullable = true)
-- IsArrDelayed: string (nullable = true)
-- IsDepDelayed: string (nullable = true)

In [9]:
airlines.count()

Out[17]: 1235349691

In [10]:
airlines.write.parquet("dbfs:/tmp/parquet/airlines_data_in_parquet.parquet")

In [11]:
dbutils.fs.ls('/tmp/parquet')

Out[19]: [FileInfo(path='dbfs:/tmp/parquet/airlines_data_in_parquet.parquet/', name='airlines_data_in_parquet.parquet/', size=0),
 FileInfo(path='dbfs:/tmp/parquet/airlines_data_parquet.parquet/', name='airlines_data_parquet.parquet/', size=0),
 FileInfo(path='dbfs:/tmp/parquet/airlines_data_parquet2.parquet/', name='airlines_data_parquet2.parquet/', size=0),
 FileInfo(path='dbfs:/tmp/parquet/airlines_parquet.parquet/', name='airlines_parquet.parquet/', size=0)]

In [12]:
airlines_df = spark.read.parquet("dbfs:/tmp/parquet/airlines_data_in_parquet.parquet")
display(airlines_df)


Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
2006,2,12,7,625,622,744,746,OO,6612,N746SK,79,84,55,-2,3,SLC,DEN,391,8,16,0,null,0,0,0,0,0,0,NO,YES
2006,2,12,7,1506,1457,1736,1737,OO,6614,N771SK,90,100,75,-1,9,DEN,MCI,533,4,11,0,null,0,0,0,0,0,0,NO,YES
2006,2,12,7,1148,1151,1330,1347,OO,6615,N910SW,102,116,89,-17,-3,DEN,GTF,624,4,9,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,1445,1454,1623,1645,OO,6615,N910SW,98,111,84,-22,-9,GTF,DEN,624,7,7,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,1013,1003,1258,1255,OO,6616,N710SK,105,112,90,3,10,DEN,CID,692,4,11,0,null,0,0,0,0,0,0,YES,YES
2006,2,12,7,1017,1021,1251,1250,OO,6618,N969SW,94,89,77,1,-4,DEN,FSD,483,4,13,0,null,0,0,0,0,0,0,YES,NO
2006,2,12,7,847,845,947,954,OO,6619,N958SW,120,129,103,-7,2,DEN,PSP,776,4,13,0,null,0,0,0,0,0,0,NO,YES
2006,2,12,7,1020,1024,1320,1325,OO,6619,N958SW,120,121,104,-5,-4,PSP,DEN,776,7,9,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,907,910,1131,1142,OO,6620,N752SK,84,92,65,-11,-3,DEN,TUL,541,5,14,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,1737,1735,1841,1843,OO,6621,N978SW,124,128,108,-2,2,DEN,PSP,776,4,12,0,null,0,0,0,0,0,0,NO,YES


In [13]:
airlines_df.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: integer (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: integer (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: string (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: integer (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- AirTime: integer (nullable = true)
-- ArrDelay: integer (nullable = true)
-- DepDelay: integer (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: integer (nullable = true)
-- TaxiOut: integer (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: integer (nullable = true)
-- WeatherDelay: integer (nullable = true)
-- NASDelay: integer (nullable = true)
-- SecurityDelay: integer (nullable = true)
-- LateAircraftDelay: integer (nullable = true)
-- IsArrDelayed: string (nullable = true)
-- IsDepDelayed: string (nullable = true)

In [14]:
airlines_df.count()

Out[24]: 1235349691

In [15]:
import numpy as np
import matplotlib.pyplot as mplt
import matplotlib.ticker as mtick

def plotHistogramData(data):
    binSides, binCounts = data

    N = len(binCounts)
    ind = np.arange(N)
    width = 1

    fig, ax = mplt.subplots()
    rects1 = ax.bar(ind+0.5, binCounts, width, color='b')

    ax.set_ylabel('Frequencies')
    ax.set_title('Histogram')
    ax.set_xticks(np.arange(N+1))
    ax.set_xticklabels(binSides)
    ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
    ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))

    mplt.show()
#plotHistogramData(airlines_first.select('Distance').rdd.flatMap(lambda x: x[0]).histogram(11))    

In [16]:
airlines_first = spark.read.option("header", "true").csv("dbfs:/databricks-datasets/airlines/part-00000")
print(airlines_first.count())

In [17]:

airlines_test = sqlContext.read.option("header", "true").csv("dbfs:/databricks-datasets/airlines/part-00000")
airlines_test.registerTempTable("airlines_test")
airlines_test.cache()


airports_count = sqlContext.sql("select count(*) from airlines_test")
airports_count.registerTempTable("airports_count")
airports_count.cache()
sqlContext.sql("select count(*) as Total_Records from airlines_test").show()

In [18]:
display(airports_count)

In [19]:
#combine all partitions together ONLY RUN IF YOU HAVE TIME ON YOUR HANDS!
airlines_all = airlines_first
for item in dbutils.fs.ls(DATA_PATH):
  if re.search(r"part-\d*", item.path):
    if item.path != "dbfs:/databricks-datasets/airlines/part-00000":
      airlines_each = spark.read.option("header", "false").csv(item.path)
      airlines_all = airlines_all.union(airlines_each)

In [20]:
airlines_all.registerTempTable("airlines_all")
airlines_all.cache()

In [21]:
airports_all_count = sqlContext.sql("select count(*) from airlines_all")
airports_all_count.registerTempTable("airports_all_count")
airports_all_count.cache()

In [22]:
display(airports_all_count)

In [23]:
sqlContext.sql("select count(*) from airlines_all").show()